<a href="https://colab.research.google.com/github/GurkiratSarna/CE888-Decision-Making-UoE/blob/CE888--Assignment-1/CE888_Assignment_1_Project_3_1900690.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Example GameState classes for Nim, OXO and Othello are included to give some idea of how you
# can write your own GameState use UCT in your 2-player game. Change the game to be played in 
# the UCTPlayGame() function at the bottom of the code.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 

from math import *
import random
import pandas as pd #To convert the final dataset into a dataframe.

class OXOState:
    """ A state of the game, i.e. the game board.
        Squares in the board are in this arrangement
        012
        345
        678
        where 0 = empty, 1 = player 1 (X), 2 = player 2 (O)
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is 2 whereas player 1 has the first move.
        self.board = [0,0,0,0,0,0,0,0,0] # 0 = empty, 1 = player 1, 2 = player 2. This is the initial board state - all positions are empty.
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = OXOState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        return st

    def DoMove(self, move):
        """ Update the state board by replacing 0 with the player playing the move at the position/move of the board.
            Must update playerJustMoved.
        """
        assert move >= 0 and move <= 8 and move == int(move) and self.board[move] == 0
        self.playerJustMoved = 3 - self.playerJustMoved
        self.board[move] = self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state. That is return all the positional values of the zroes in the state board.
        """
        return [i for i in range(9) if self.board[i] == 0]
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        for (x,y,z) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]: #Winning possibilities of the board
            if self.board[x] == self.board[y] == self.board[z]: # check if values in all the 3 positions is of the same player
                # check if the player that just moved is same as the value in the winning positions, if yes return 1 else 0 stating that the other player wins.
                if self.board[x] == playerjm: 
                    return 1.0
                else:
                    return 0.0
        if self.GetMoves() == []:
          return 0.5 # draw
        return False # Should not be possible to get here

    def __repr__(self): # This is how the return value is defined for this class.
        s= ""
        for i in range(9): 
            s += ".XO"[self.board[i]] # . for 0, X for 1 and O for 2 positional values
            if i % 3 == 2: s += "\n"
        return s


class Node:
    """ A node in the game tree. Note : wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        #parentNode stores all the paarents from the rootnode until the current node for backpropogation which then deletes until it is None.
        self.parentNode = parent # "None" for the root node.
        self.childNodes = []
        self.wins = 0
        self.visits = 0 #The number of itermax passed
        self.untriedMoves = state.GetMoves() # future child nodes. The available positions to be played at any point of the game.
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node.
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result

    def __repr__(self): # Added other variables to be retiurned to check the flow of the variable during testing small iterations.
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + " PJM:" + str(self.playerJustMoved) + "]"

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
             s += c.TreeToString(indent+1)
        return s

    def IndentString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
             s += str(c) + "\n"
        return s


def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree. This updates the parent node as well.

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function.
        # This loop will play a game within the random move picked up before and then return the best move for one chance of the actual game being played.
        while state.GetMoves() != []: # while state is non-terminal
            #print("IN ROLLOUT")
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    #if verbose: print(rootnode.TreeToString(0))
    #else: print(rootnode.ChildrenToString())

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited as the best move.
                
def UCTPlayGame():
    """ Play a sample game between two UCT players where each player gets a different number 
        of UCT iterations (= simulations = tree nodes).
    """
    state = OXOState()

    tempdataset=[[0,0,0,0,0,0,0,0,0]] # LINE ADDED. A temporary datset which will accumulate all the stages of one game
    # A list of best moves for that particular game. This will always have length of tempdataset-1 because of the initial stage in the temporary dataset.
    bestmovelist=[] # LINE ADDED.
    pjm_list=[] #List of player moving at the current board
    while state.GetMoves() != []:
        #print(str(state)) # prints the board - Uncomment
        if state.playerJustMoved == 1:
            # Biased towards one player so that someone wins. 
            #1000:2000 -> No one wins; 1500:2000 -> No one wins; 10:1000 -> nearly 2.5% winning
            #10:1000 -> with 5000 iterations of UCTPlaygame -> 12.5% winning games
            m = UCT(rootstate=state, itermax=10, verbose=False)  # play with values for itermax and verbose = True.
            #print("In 10 itermax")
        else:
            m = UCT(rootstate=state, itermax=1000, verbose=False) # itermax decides the total value of visits to one node in a given game.
            #print("In 5 itermax")
        #print("Best Move: " + str(m) + "\n") # Uncomment
        state.DoMove(m)
        

        pjm_list.append(state.playerJustMoved)
        bestmovelist.append(m) # LINE ADDED. 
        tempdataset.append(list(state.board))  #LINE ADDED.
        
        if state.GetResult(state.playerJustMoved) != False:
            #print(str(state)) # Uncomment
            break
    
    if state.GetResult(state.playerJustMoved) == 1.0:
        print("Player " + str(state.playerJustMoved) + " wins!")
        winner=state.playerJustMoved
    elif state.GetResult(state.playerJustMoved) == 0.0:
        print("Player " + str(3 - state.playerJustMoved) + " wins!")
        winner=state.playerJustMoved
    else: 
      print("Nobody wins!")
      winner=0

    # This list will always have one element less compared to tempdataset because the last state of the game will never have any move associated to it.
    # Becasue we are combining the two lists so we want them of equal length hence the move of the last stage will be NaN, 
    # the last game state will be removed later from the final dataframe.
    bestmovelist.append('NaN')  # LINE ADDED.
    pjm_list.append('NaN')

    for i in range(len(tempdataset)): # LINE ADDED
      tempdataset[i].append(pjm_list[i]) #Append the Player at the board state
      tempdataset[i].append(bestmovelist[i]) # LINE ADDED. Append the move taken at every stage.
    tempdataset
    return tempdataset, winner # LINE ADDED.

if __name__ == "__main__":
    """ Play a single game to the end using UCT for both players. 
    """
    finaldataset=[] # LINE ADDED. A list os lists of the game stages. Each list is one game, representing the lists of stages returned.
    finaldataset1=[] # LINE ADDED. A list of all the stages of the games in the finaldataset. It is the output of converting a list of list into one list.
    
    p1win=0
    p2win=0
    Nowin=0
    winner=0
    for i in range(5000): # LINE ADDED. Run the UCTPlayGame 2000 or 5000 times.
      returnds, winner =UCTPlayGame() # LINE EDITED. returnds stores the game temporarily.
      finaldataset.append(returnds) # LINE ADDED. 
      #print("Winner : ", winner)
      if winner==1:
        p1win+=1
      elif winner==2:
        p2win+=1
      else:
        Nowin+=1

    #Convert list of lists to one list
    for i in finaldataset:  # LINE ADDED
      for j in i:  # LINE ADDED
        finaldataset1.append(j)  # LINE ADDED
    
    #Name the columns of the finaldataset1 and convert it into a dataframe. # LINE ADDED
    datatbp=pd.DataFrame(finaldataset1, columns=['0th pos', '1st pos', '2nd pos', '3rd pos', '4th pos', '5th pos', '6th pos', '7th pos', '8th pos', 'Player','Move'])
 
    #Remove the rows that have 'NaN' as these are the rows depicting the last stage of the game with no move.
    datatbp.drop(datatbp[datatbp.Move == 'NaN'].index, inplace=True)
    datatbp.reset_index(drop=True, inplace=True) # LINE ADDED. Reset the row index of the data frame
    #print(datatbp)
    print("Wins : \n Player1 : ", p1win, " Player2 : ", p2win, " Nobody wins : ", Nowin)


...
...
...

...
X..
...

...
X..
.O.

X..
X..
.O.

X..
XO.
.O.

X..
XO.
XO.

Player 1 wins!
...
...
...

X..
...
...

XO.
...
...

XO.
.X.
...

XOO
.X.
...

XOO
.X.
.X.

XOO
OX.
.X.

XOO
OX.
.XX

Player 1 wins!
Wins : 
 Player1 :  2  Player2 :  0  Nobody wins :  0


In [6]:
datatbp

,0th pos,1st pos,2nd pos,3rd pos,4th pos,5th pos,6th pos,7th pos,8th pos,Player,Move
0,0,0,0,0,0,0,0,0,0,1,3
1,0,0,0,1,0,0,0,0,0,2,7
2,0,0,0,1,0,0,0,2,0,1,0
3,1,0,0,1,0,0,0,2,0,2,4
4,1,0,0,1,2,0,0,2,0,1,6
5,0,0,0,0,0,0,0,0,0,1,0
6,1,0,0,0,0,0,0,0,0,2,1
7,1,2,0,0,0,0,0,0,0,1,4
8,1,2,0,0,1,0,0,0,0,2,2
9,1,2,2,0,1,0,0,0,0,1,7


In [0]:
#Download the Dataset to be processed in Assignment 2
#2000*10= 20000; 20000-2000=18000 rows if nobody wins for all games.
from google.colab import files
datatbp.to_csv('CE888_DataToBeProcessed_1900690.csv')
files.download('CE888_DataToBeProcessed_1900690.csv')

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from sklearn import tree
import numpy as np
import matplotlib.pyplot as plt #plotting charts
import pandas as pd #helps importing datasets

In [0]:
file_id = r'C:\Users\DELL\OneDrive - University of Essex\CE888 - Decision Making\CE888_DataToBeProcessed_1900690_NobodyWins_2.csv'
downloaded = drive.CreateFile({'id': file_id})
downloaded = drive.CreateFile({'id':'C:\Users\DELL\OneDrive - University of Essex\CE888 - Decision Making\CE888_DataToBeProcessed_1900690_NobodyWins_2.csv'}) # replace the id with id of file you want to access
downloaded.GetContentFile('xyz.csv')  

# Read file as panda dataframe
xyz = pd.read_csv('xyz.csv') 

SyntaxError: ignored